In [2]:
import efficientnet.keras as efn

from keras import layers, Model
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [3]:
!nvidia-smi

Sun Jun 21 13:21:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P6000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   20C    P8     8W / 250W |      1MiB / 24449MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
INPUT_SHAPE = (300, 300, 3)

In [5]:
NUM_CLASSES = 2 # binary classification

In [6]:
efn_conv_base = efn.EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_tensor=layers.Input(shape=INPUT_SHAPE)
)

In [7]:
# base_model.summary()

In [8]:
class FCModule:
    @staticmethod
    def build(body, num_classes):
        head = body.output
        head = layers.Flatten(name='flatten_vec')(head)
        head = layers.Dense(256, activation='relu')(head)
        head = layers.Dropout(0.5)(head)
        
        head = layers.Dense(num_classes, activation='softmax')(head)
        
        return head

In [9]:
FC = FCModule.build(body=efn_conv_base, num_classes=NUM_CLASSES)

In [10]:
model = Model(inputs=efn_conv_base.input, outputs=FC)

In [11]:
# model.summary()

In [12]:
for layer in efn_conv_base.layers:
    layer.trainable = False

In [13]:
from h5imagegenerator import HDF5ImageGenerator

In [14]:
from albumentations import (
    Compose, ShiftScaleRotate
)

In [15]:
train_gen = HDF5ImageGenerator(
    src= './train.h5',
    scaler=True,
    labels_encoding=False,
    batch_size=32)

val_gen = HDF5ImageGenerator(
    src= './val.h5',
    scaler=True,
    labels_encoding=False,
    batch_size=32)

In [14]:
rms = RMSprop(learning_rate=0.001, rho=0.9)

model.compile(
    loss='binary_crossentropy',
    metrics=['accuracy'],
    optimizer=rms
)

h = model.fit(
    train_gen,
    validation_data=val_gen,
    verbose=1,
    epochs=30
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
  8/456 [..............................] - ETA: 59:05 - loss: 6.8608 - accuracy: 0.5146

KeyboardInterrupt: 